1.	Consider the dataset Dataset/Housing.csv with y=price as response variable. Try the following stacking(StackingRegressor) options with parameters of your choice for scoring as R2 score:
a.	Base estimators: KNN regressor, Decision Tree Regressor, Elasticnet
b.	Final Estimator: Gradient Boosting


In [31]:
import pandas as pd 
import matplotlib.pyplot as plt 
#from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np 
from sklearn.linear_model import ElasticNet
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

hd = pd.read_csv("/Users/darshmac/Documents/cdac/For DBDA/Advance Analytics/Datasets/Housing.csv")
hd

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...
541,91500.0,4800,3,2,4,yes,yes,no,no,yes,0,no
542,94000.0,6000,3,2,4,yes,no,no,no,yes,0,no
543,103000.0,6000,3,2,4,yes,yes,no,no,yes,1,no
544,105000.0,6000,3,2,2,yes,yes,no,no,yes,1,no


In [17]:
dum_hd = pd.get_dummies(hd, drop_first=True)
X = dum_hd.drop('price', axis=1)
y = dum_hd['price']
#we will tune model lets not train test split
#X_train, X_test, y_train, y_test = train_test_split(X,y, 
#                               test_size=0.3,
#                              stratify=y,
#                              random_state=23)

In [47]:
X

,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,5850,3,1,2,1,1,0,1,0,0,0
1,4000,2,1,1,0,1,0,0,0,0,0
2,3060,3,1,1,0,1,0,0,0,0,0
3,6650,3,1,2,0,1,1,0,0,0,0
4,6360,2,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
541,4800,3,2,4,0,1,1,0,0,1,0
542,6000,3,2,4,0,1,0,0,0,1,0
543,6000,3,2,4,1,1,1,0,0,1,0
544,6000,3,2,2,1,1,1,0,0,1,0


In [32]:
#lets choose different model
kn = KNeighborsRegressor()
dtr = DecisionTreeRegressor(random_state=23)
en = ElasticNet()
gb = GradientBoostingRegressor(random_state=23)

In [48]:
#creating stack for models to fit
stack = StackingRegressor(estimators=[('KN', kn),
                                       ('dtr', dtr),
                                     ('EN',en)],
                           final_estimator=gb)

#stack.fit(X_train, y_train) # all train set operations
#y_pred = stack.predict(X_test) # all test operations
#print(accuracy_score(y_test, y_pred))

In [49]:
stack.get_params()

{'cv': None,
 'estimators': [('KN', KNeighborsRegressor()),
  ('dtr', DecisionTreeRegressor(random_state=23)),
  ('EN', ElasticNet())],
 'final_estimator__alpha': 0.9,
 'final_estimator__ccp_alpha': 0.0,
 'final_estimator__criterion': 'friedman_mse',
 'final_estimator__init': None,
 'final_estimator__learning_rate': 0.1,
 'final_estimator__loss': 'squared_error',
 'final_estimator__max_depth': 3,
 'final_estimator__max_features': None,
 'final_estimator__max_leaf_nodes': None,
 'final_estimator__min_impurity_decrease': 0.0,
 'final_estimator__min_samples_leaf': 1,
 'final_estimator__min_samples_split': 2,
 'final_estimator__min_weight_fraction_leaf': 0.0,
 'final_estimator__n_estimators': 100,
 'final_estimator__n_iter_no_change': None,
 'final_estimator__random_state': 23,
 'final_estimator__subsample': 1.0,
 'final_estimator__tol': 0.0001,
 'final_estimator__validation_fraction': 0.1,
 'final_estimator__verbose': 0,
 'final_estimator__warm_start': False,
 'final_estimator': GradientB

In [50]:
params = {'KN__n_neighbors':[5,10,15],
         'dtr__max_depth':[3,5],
         'final_estimator__max_features':[2,5,10],
         'passthrough':[True,False]}

In [37]:
from sklearn.model_selection import GridSearchCV,KFold

In [38]:
kflod = KFold(n_splits=5, shuffle=True,random_state=23)

In [51]:
gcv = GridSearchCV(stack,param_grid=params,cv=kflod,verbose=3)

In [52]:
gcv.fit(X,y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END KN__n_neighbors=5, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.661 total time=   0.2s
[CV 2/5] END KN__n_neighbors=5, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.597 total time=   0.2s
[CV 3/5] END KN__n_neighbors=5, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.657 total time=   0.1s
[CV 4/5] END KN__n_neighbors=5, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.558 total time=   0.2s
[CV 5/5] END KN__n_neighbors=5, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.554 total time=   0.2s
[CV 1/5] END KN__n_neighbors=5, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.521 total time=   0.2s
[CV 2/5] END KN__n_neighbors=5, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.611 total time=   0.2s
[CV 3

[CV 2/5] END KN__n_neighbors=10, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.650 total time=   0.2s
[CV 3/5] END KN__n_neighbors=10, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.669 total time=   0.1s
[CV 4/5] END KN__n_neighbors=10, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.568 total time=   0.2s
[CV 5/5] END KN__n_neighbors=10, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.545 total time=   0.1s
[CV 1/5] END KN__n_neighbors=10, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.406 total time=   0.2s
[CV 2/5] END KN__n_neighbors=10, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.645 total time=   0.2s
[CV 3/5] END KN__n_neighbors=10, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.583 total time=   0.2s
[CV 4/5] END KN__n_neighbors=10, dtr__max_depth=3, final_es

[CV 4/5] END KN__n_neighbors=15, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.550 total time=   0.2s
[CV 5/5] END KN__n_neighbors=15, dtr__max_depth=3, final_estimator__max_features=2, passthrough=True;, score=0.534 total time=   0.2s
[CV 1/5] END KN__n_neighbors=15, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.479 total time=   0.2s
[CV 2/5] END KN__n_neighbors=15, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.570 total time=   0.2s
[CV 3/5] END KN__n_neighbors=15, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.537 total time=   0.2s
[CV 4/5] END KN__n_neighbors=15, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.502 total time=   0.2s
[CV 5/5] END KN__n_neighbors=15, dtr__max_depth=3, final_estimator__max_features=2, passthrough=False;, score=0.478 total time=   0.2s
[CV 1/5] END KN__n_neighbors=15, dtr__max_depth=3, final_

GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=True),
             estimator=StackingRegressor(estimators=[('KN',
                                                      KNeighborsRegressor()),
                                                     ('dtr',
                                                      DecisionTreeRegressor(random_state=23)),
                                                     ('EN', ElasticNet())],
                                         final_estimator=GradientBoostingRegressor(random_state=23)),
             param_grid={'KN__n_neighbors': [5, 10, 15],
                         'dtr__max_depth': [3, 5],
                         'final_estimator__max_features': [2, 5, 10],
                         'passthrough': [True, False]},
             verbose=3)

In [53]:
gcv.best_params_

{'KN__n_neighbors': 10,
 'dtr__max_depth': 5,
 'final_estimator__max_features': 2,
 'passthrough': True}

In [54]:
gcv.best_score_

0.6266880934662029